In [1]:
## INPUT
name="DescentIntoFuckery"

In [35]:
import pandas as pd
import ast
import datetime
import re
from bs4 import BeautifulSoup

skill_labels=['ACROBATICS',
 'ANIMAL HANDLING',
 'ARCANA',
 'ATHLETICS',
 'DECEPTION',
 'HISTORY',
 'INSIGHT',
 'INTIMIDATION',
 'INVESTIGATION',
 'MEDICINE',
 'NATURE',
 'PERCEPTION',
 'PERFORMANCE',
 'PERSUASION',
 'RELIGION',
 'SLEIGHT OF HAND',
 'STEALTH',
 'SURVIVAL']

attr_labels=['STRENGTH',
 'DEXTERITY',
 'CONSTITUTION',
 'INTELLIGENCE',
 'WISDOM',
 'CHARISMA']

save_labels=['STRENGTH SAVE',
 'DEXTERITY SAVE',
 'CONSTITUTION SAVE',
 'INTELLIGENCE SAVE',
 'WISDOM SAVE',
 'CHARISMA SAVE',
 'DEATH SAVE']

# read player names and get all player names
file = open("players/players_"+name+".txt", "r")
contents = file.read()
players = ast.literal_eval(contents)
file.close()
plyrs=list(set(players.values()))

print("Loading chat log...")
soup = BeautifulSoup(open('raw/'+name+'.html', 'r').read(), 'html.parser')
msgs=soup.findAll('div',{'class' : re.compile('message \w+')})

print("Parsing messages...")
plyr=None
when=None
df = pd.DataFrame(columns=['Player','NPC','Timestamp','Skill/Weapon/Spell','Result', 'Raw','Plus'])
for c,m in enumerate(msgs):
    if c % 500 == 0:
        print(c+1,"/",len(msgs))
    
    ### get player and timestamp
    npc=False if m.findAll('div',{'class': ['sheet-rolltemplate-npc','sheet-rolltemplate-npcfullatk']})==[] else True
    who=m.findAll('span',{'class': 'by'})
    tmpplyr=None
    if (who!=[] and not npc) or (m.find_all('div',{'class': ["sheet-row sheet-subheader"]})!=[] and npc):
        plyr=m.findAll('span',{'class': 'by'})[0].renderContents() if not npc or m.find_all('div',{'class': ["sheet-row sheet-subheader"]})==[] else m.find_all('div',{'class': ["sheet-row sheet-subheader"]})[0].find('span',{'class': ["sheet-italics"]}).renderContents()
        when=m.find('span',{'class': 'tstamp'}).renderContents() if not m.find('span',{'class': 'tstamp'})==None else when
    elif npc and m.find_all('div',{'class': ["sheet-row sheet-subheader"]})==[]: 
        tmpplyr="MISSING" if name!="DescentIntoFuckery" else "Philaster Fisk"
        
    log=True
    mcontent=m.findAll('div',{'class': ['sheet-rolltemplate-skill','sheet-rolltemplate-simple','sheet-rolltemplate-npc','sheet-rolltemplate-atk','sheet-rolltemplate-npcfullatk']})
    parsing=None
    if mcontent!=[]:
        parsing="Attack" if m.findAll('div',{'class': ['sheet-rolltemplate-atk','sheet-rolltemplate-npcfullatk']})!=[] else "Skill"
    
    if parsing!=None:
        rolls=mcontent[0].find_all('div',{'class': ['sheet-roll','sheet-adv','sheet-solo']})
        if len(rolls)==1:
            roll=rolls[0]
        elif len(rolls)==2:
            roll=rolls[0] if rolls[1].find('span',{'class': 'sheet-grey'})!=None else rolls[1]
        elif not npc:
            log=False
        result=roll.find('span',{'class': lambda x: x and 'inlinerollresult' in x}) if not npc else mcontent[0].find('span',{'class': lambda x: x and 'inlinerollresult' in x})        
        if result!=None:
            result=result.renderContents()
        else:
            log=False
        if not npc:
            tmplabel=mcontent[0].find('div',{'class': 'sheet-label'}).renderContents()
            if rolls==[]:
                label=["",""]
            elif m.findAll('div',{'class': ['sheet-rolltemplate-atk']})!=[]:
                label=[re.sub("</*[a-zA-Z]+","",str(tmplabel).split(">")[2]),re.sub("[^0-9-+.]","",str(tmplabel).split(">")[4])] 
            else:
                label=[re.sub("</*[a-zA-Z]+","",str(tmplabel).split(">")[1]),re.sub("[^0-9-+.]","",str(tmplabel).split(">")[2])]
            label[1]=eval(label[1]) if label[1]!='' else "0"
        else:
            if mcontent[0].findAll(attrs={'class': re.compile(r"^sheet-row$")})[-1].find("span", {"class":None})!=None:
                tmplabel=str(mcontent[0].findAll(attrs={'class': re.compile(r"^sheet-row$")})[0])+str(mcontent[0].findAll(attrs={'class': re.compile(r"^sheet-row$")})[-1].find("span", {"class":None}).find('span',{'class': lambda x: x and 'inlinerollresult' in x}))
                if re.search('\+\(?([A-Za-z0-9+-]+)\)?', str(mcontent[0].findAll(attrs={'class': re.compile(r"^sheet-row$")})[-1].find('span',{'class': lambda x: x and 'inlinerollresult' in x})))!=None:
                    label= [mcontent[0].findAll(attrs={'class': re.compile(r"^sheet-row$")})[0].find("span", {"class":None}).renderContents().decode('UTF-8').upper(), eval(re.sub("[\(\)]","",re.search('\+\(?([A-Za-z0-9+-]+)\)?', str(mcontent[0].findAll(attrs={'class': re.compile(r"^sheet-row$")})[-1].find('span',{'class': lambda x: x and 'inlinerollresult' in x})))[0]))]
                else:
                    label= [mcontent[0].findAll(attrs={'class': re.compile(r"^sheet-row$")})[0].find("span", {"class":None}).renderContents().decode('UTF-8').upper(), -100]                
            else:
                log=False
                
        if  log and (label[0] not in ["F","Chromatic Orb","D S","D K","T T","INITIATIVE","","D","S","C","P","H K","P S","F K","(+3)","HIT POINTS"]) and ("HIT DICE" not in label[0]) and (label[0] not in attr_labels):
            if tmpplyr==None:
                df.loc[str(c), 'Player'] = re.sub(':','',plyr.decode('UTF-8')) if type(plyr)!=str else plyr
            else:
                df.loc[str(c), 'Player'] = tmpplyr
            df.loc[str(c), 'NPC'] = 1 if npc else 0
            df.loc[str(c), 'Timestamp'] = when.decode('UTF-8') if type(when)!=str else when
            df.loc[str(c), 'Skill/Weapon/Spell'] = str(label[0])
            df.loc[str(c), 'Result'] = result.decode('UTF-8')
            df.loc[str(c), 'Raw'] = str(int(float(result))-int(label[1]))
            df.loc[str(c), 'Plus'] = str(label[1])
            df.loc[str(c), 'Final'] = 1
            df.loc[str(c), 'Type'] = parsing
            if len(rolls)==2:
                if not npc:
                    rollgrey=rolls[1] if rolls[1].find('span',{'class': 'sheet-grey'})!=None else rolls[0]
                else:
                    mcontent[0].findAll(attrs={'class': re.compile(r"^sheet-row$")})[-1].find("span", {"class":"sheet-grey"})
                resultgrey=rollgrey.find('span',{'class': lambda x: x and 'inlinerollresult' in x}).renderContents()
                if tmpplyr==None:
                    df.loc[str(c)+".2", 'Player'] = re.sub(':','',plyr.decode('UTF-8')) if type(plyr)!=str else plyr
                else:
                    df.loc[str(c), 'Player'] = tmpplyr
                df.loc[str(c)+".2", 'NPC'] = 1 if npc else 0
                df.loc[str(c)+".2", 'Timestamp'] = when.decode('UTF-8') if type(when)!=str else when
                df.loc[str(c)+".2", 'Skill/Weapon/Spell'] = str(label[0])
                df.loc[str(c)+".2", 'Result'] = resultgrey.decode('UTF-8')
                df.loc[str(c)+".2", 'Raw'] = str(int(float(resultgrey))-int(label[1]))
                df.loc[str(c)+".2", 'Plus'] = str(label[1])
                df.loc[str(c)+".2", 'Final'] = 0
                df.loc[str(c)+".2", 'Type'] = parsing

df["Skill/Weapon/Spell"]=df["Skill/Weapon/Spell"].str.strip()
df["Timestamp"]=pd.to_datetime(df["Timestamp"])
df.loc[df.Timestamp.dt.hour<6,"Timestamp"] = df.loc[df.Timestamp.dt.hour<6,"Timestamp"] + datetime.timedelta(days=1)
df.loc[df[df["NPC"]==0].index,'Player']=df.loc[df[df["NPC"]==0].index,'Player'].apply(lambda x: players[x])
df['Day'] = df['Timestamp'].dt.strftime('%d.%m.%Y')
df.loc[(df["Type"]=="Skill")&(df["Skill/Weapon/Spell"].isin(skill_labels)==False)&(df["Skill/Weapon/Spell"].str.contains("SAVE")),"Type"]="Save"
df.loc[(df["Type"]=="Skill")&(df["Skill/Weapon/Spell"].isin(skill_labels)==False)&(df["Skill/Weapon/Spell"].str.contains("SAVE")==False),"Type"]="Other"
df.to_csv("rolls_"+name+".csv",index=False,sep=";")
df

Loading chat log...
Parsing messages...
1 / 3399
501 / 3399
1001 / 3399
1501 / 3399
2001 / 3399
2501 / 3399
3001 / 3399


,Player,NPC,Timestamp,Skill/Weapon/Spell,Result,Raw,Plus,Final,Type,Day
10,Smallfolk,0,2021-01-03 19:36:00,STEALTH,25,19,6,1.0,Skill,03.01.2021
11,Kallistē,0,2021-01-03 19:38:00,DECEPTION,20,17,3,1.0,Skill,03.01.2021
11.2,Kallistē,0,2021-01-03 19:38:00,DECEPTION,11,8,3,0.0,Skill,03.01.2021
12,Bethrynne,0,2021-01-03 19:50:00,HISTORY,8,2,6,1.0,Skill,03.01.2021
13,Smallfolk,0,2021-01-03 19:50:00,HISTORY,12,12,0,1.0,Skill,03.01.2021
...,...,...,...,...,...,...,...,...,...,...
3389,Vampire,1,2021-04-25 23:19:00,DEXTERITY SAVE,16,7,9,1.0,Save,25.04.2021
3392,Vampire Spawn,1,2021-04-25 23:19:00,DEXTERITY SAVE,22,16,6,1.0,Save,25.04.2021
3393,Vampire Spawn,1,2021-04-25 23:19:00,DEXTERITY SAVE,25,19,6,1.0,Save,25.04.2021
3394,Smallfolk,0,2021-04-25 23:30:00,Smith's Tools,23,15,8,1.0,Other,25.04.2021


In [36]:
df[df["Type"]=="Other"]

,Player,NPC,Timestamp,Skill/Weapon/Spell,Result,Raw,Plus,Final,Type,Day
71,Bethrynne,0,2021-01-10 19:55:00,INITIATIVE,4,1,3,1.0,Other,10.01.2021
72,Smallfolk,0,2021-01-10 20:04:00,INITIATIVE,14,10,4,1.0,Other,10.01.2021
73,Kallistē,0,2021-01-10 20:04:00,INITIATIVE,12,9,3,1.0,Other,10.01.2021
184,Smallfolk,0,2021-01-10 23:38:00,INITIATIVE,10,6,4,1.0,Other,10.01.2021
185,Kallistē,0,2021-01-10 23:38:00,INITIATIVE,22,18,4,1.0,Other,10.01.2021
...,...,...,...,...,...,...,...,...,...,...
3283,Bethrynne,0,2021-04-25 18:01:00,INITIATIVE,5,2,3,1.0,Other,25.04.2021
3284,Kallistē,0,2021-04-25 18:01:00,INITIATIVE,21,17,4,1.0,Other,25.04.2021
3286,Smallfolk,0,2021-04-25 18:01:00,INITIATIVE,14.2,9,5.2,1.0,Other,25.04.2021
3394,Smallfolk,0,2021-04-25 23:30:00,Smith's Tools,23,15,8,1.0,Other,25.04.2021


In [37]:
df[(df["Raw"].astype("int64")<1) | (df["Raw"].astype("int64")>20)]

,Player,NPC,Timestamp,Skill/Weapon/Spell,Result,Raw,Plus,Final,Type,Day
